In [2]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import shutil
import time

from api import *
from dataframe import *
from database import *
from file import *
from sql import *
from tables import *

# cfg

In [3]:
cfg = {
    "host": f"192.168.0.144",
    "user": f"{os.environ['POSTGRE_USER']}",
    "password": f"{os.environ['POSTGRE_PASSWORD']}",
    "port": "5432",
    "database": "herbario"
}
filename = "csv/original.csv"
filename_george = "csv/dados-george.csv"

engine, session = connect(cfg)

2022-11-21 20:56:34,492 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2022-11-21 20:56:34,492 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-11-21 20:56:34,494 INFO sqlalchemy.engine.Engine select current_schema()
2022-11-21 20:56:34,494 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-11-21 20:56:34,495 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2022-11-21 20:56:34,496 INFO sqlalchemy.engine.Engine [raw sql] {}


# Load data

In [4]:
dataframe = pandas.read_csv("csv/original.csv", sep=";", low_memory=False, skipinitialspace=True)
dataframe_george = pandas.read_csv(filename_george, sep=";", low_memory=False, skipinitialspace=True)

In [5]:
create_table_if_not_exists(cfg, engine, "county")
create_table_if_not_exists(cfg, engine, "data")

2022-11-21 20:56:35,190 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where n.nspname=%(schema)s and relname=%(name)s
2022-11-21 20:56:35,191 INFO sqlalchemy.engine.Engine [generated in 0.00101s] {'schema': 'herbario', 'name': 'county'}
2022-11-21 20:56:35,193 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-21 20:56:35,195 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-11-21 20:56:35,195 INFO sqlalchemy.engine.Engine [generated in 0.00087s] {'name': 'data'}
2022-11-21 20:56:35,197 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-11-21 20:56:35,197 INFO sqlalchemy.engine.Engine [cached since 0.002736s ago] {'name': 'county'}
2022-11-21 20:56:35,198 INFO sqlalchemy.engine.E

In [6]:
if session.query(County).count() == 0:
    response = get_municipies()
    for i, county in enumerate(response.json()):
        session.add(create_county(county))
        make_operation(session)

2022-11-21 20:56:35,221 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-21 20:56:35,224 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT county.id AS county_id, county.county AS county_county, county.uf AS county_uf, county.uf_name AS county_uf_name 
FROM county) AS anon_1
2022-11-21 20:56:35,225 INFO sqlalchemy.engine.Engine [generated in 0.00079s] {}


In [7]:
if session.query(DataSP).count() == 0:
    change_header(dataframe)
    dataframe = preprocess(dataframe)
    # dataframe.iloc[24230]
    for row in dataframe.iterrows():
        session.add(create_datasp(row[1]))
        make_operation(session)

2022-11-21 20:56:35,251 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT data.seq AS data_seq, data.modified AS data_modified, data.institution_code AS data_institution_code, data.collection_code AS data_collection_code, data.catalog_number AS data_catalog_number, data.basis_of_record AS data_basis_of_record, data.kingdom AS data_kingdom, data.phylum AS data_phylum, data.classe AS data_classe, data."order" AS data_order, data.family AS data_family, data.genus AS data_genus, data.specific_epithet AS data_specific_epithet, data.infraspecific_epithet AS data_infraspecific_epithet, data.scientific_name AS data_scientific_name, data.scientific_name_authorship AS data_scientific_name_authorship, data.identified_by AS data_identified_by, data.year_identified AS data_year_identified, data.month_identified AS data_month_identified, data.day_identified AS data_day_identified, data.type_status AS data_type_status, data.recorded_by AS data_recorded_by, data.record_number AS d

In [8]:
if session.query(DataSP).filter(DataSP.george == True).count() == 0:
    for row in dataframe_george.iterrows():
        if row[1]["GEORGE"].lower() == "sim":
            session.query(DataSP).filter(DataSP.seq == row[1]["seq"]).update({"george": True}, synchronize_session=False)
            make_operation(session)

2022-11-21 20:56:35,273 INFO sqlalchemy.engine.Engine SELECT count(*) AS count_1 
FROM (SELECT data.seq AS data_seq, data.modified AS data_modified, data.institution_code AS data_institution_code, data.collection_code AS data_collection_code, data.catalog_number AS data_catalog_number, data.basis_of_record AS data_basis_of_record, data.kingdom AS data_kingdom, data.phylum AS data_phylum, data.classe AS data_classe, data."order" AS data_order, data.family AS data_family, data.genus AS data_genus, data.specific_epithet AS data_specific_epithet, data.infraspecific_epithet AS data_infraspecific_epithet, data.scientific_name AS data_scientific_name, data.scientific_name_authorship AS data_scientific_name_authorship, data.identified_by AS data_identified_by, data.year_identified AS data_year_identified, data.month_identified AS data_month_identified, data.day_identified AS data_day_identified, data.type_status AS data_type_status, data.recorded_by AS data_recorded_by, data.record_number AS d

In [9]:
from unaccent import unaccent

data_piperaceae = session.query(DataSP).all()
data_uf = session.query(sqlalchemy.func.lower(unaccent(County.uf))).distinct().all()
data_uf_name = session.query(County.uf_name, sqlalchemy.func.lower(unaccent(County.uf_name))).distinct().all()
data_county = session.query(County.county, unaccent(County.county)).filter(County.uf=="PR").all()

2022-11-21 20:56:35,309 INFO sqlalchemy.engine.Engine SELECT data.seq AS data_seq, data.modified AS data_modified, data.institution_code AS data_institution_code, data.collection_code AS data_collection_code, data.catalog_number AS data_catalog_number, data.basis_of_record AS data_basis_of_record, data.kingdom AS data_kingdom, data.phylum AS data_phylum, data.classe AS data_classe, data."order" AS data_order, data.family AS data_family, data.genus AS data_genus, data.specific_epithet AS data_specific_epithet, data.infraspecific_epithet AS data_infraspecific_epithet, data.scientific_name AS data_scientific_name, data.scientific_name_authorship AS data_scientific_name_authorship, data.identified_by AS data_identified_by, data.year_identified AS data_year_identified, data.month_identified AS data_month_identified, data.day_identified AS data_day_identified, data.type_status AS data_type_status, data.recorded_by AS data_recorded_by, data.record_number AS data_record_number, data.field_numb

/tmp/ipykernel_1248396/3996990519.py:4: SAWarning: Class unaccent will not make use of SQL compilation caching as it does not set the 'inherit_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Set this attribute to True if this object can make use of the cache key generated by the superclass.  Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  data_uf = session.query(sqlalchemy.func.lower(unaccent(County.uf))).distinct().all()


### create file meta-data imgs george

In [10]:
result = session.query(DataSP.barcode, DataSP.genus, DataSP.specific_epithet).filter(DataSP.george==True).all()

columns = ['barcode', 'genus', 'specific_epithet']
values = [row for row in result]
output = pd.DataFrame(values, columns=columns)
output.to_excel('genus-and-specific_epithet-by-image.xlsx', na_rep='', engine='xlsxwriter', index=None)
output.to_csv('genus-and-specific_epithet-by-image.csv', na_rep='')

2022-11-21 20:56:39,884 INFO sqlalchemy.engine.Engine SELECT data.barcode AS data_barcode, data.genus AS data_genus, data.specific_epithet AS data_specific_epithet 
FROM data 
WHERE data.george = true
2022-11-21 20:56:39,885 INFO sqlalchemy.engine.Engine [generated in 0.00082s] {}


2022-11-21 20:56:48,117 INFO sqlalchemy.engine.Engine UPDATE data SET index=((setweight(to_tsvector(%(to_tsvector_1)s, data.county), %(setweight_1)s) || setweight(to_tsvector(%(to_tsvector_2)s, data.state_province), %(setweight_2)s)) || setweight(to_tsvector(%(to_tsvector_3)s, data.country), %(setweight_3)s))
2022-11-21 20:56:48,118 INFO sqlalchemy.engine.Engine [generated in 0.00094s] {'to_tsvector_1': 'portuguese', 'setweight_1': 'A', 'to_tsvector_2': 'portuguese', 'setweight_2': 'B', 'to_tsvector_3': 'portuguese', 'setweight_3': 'C'}
2022-11-21 20:56:48,562 INFO sqlalchemy.engine.Engine COMMIT
2022-11-21 20:56:48,839 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-21 20:56:48,841 INFO sqlalchemy.engine.Engine SELECT data.seq AS data_seq, ts_rank(data.index, to_tsquery(%(to_tsquery_1)s, %(to_tsquery_2)s)) AS ts_rank_1 
FROM data 
WHERE data.index @@ to_tsquery(%(to_tsquery_1)s, %(to_tsquery_2)s)
2022-11-21 20:56:48,841 INFO sqlalchemy.engine.Engine [generated in 0.00063s] {'to

[(6832, 0.24317084),
 (6847, 0.24317084),
 (7049, 0.24317084),
 (7099, 0.24317084),
 (4, 0.24317084),
 (5, 0.24317084),
 (7368, 0.24317084),
 (7383, 0.24317084),
 (7553, 0.24317084),
 (14, 0.24317084),
 (15, 0.24317084),
 (7649, 0.24317084),
 (7683, 0.24317084),
 (7695, 0.24317084),
 (17, 0.24317084),
 (7705, 0.24317084),
 (7706, 0.24317084),
 (7836, 0.24317084),
 (7852, 0.24317084),
 (7853, 0.24317084),
 (7856, 0.24317084),
 (7854, 0.24317084),
 (7866, 0.24317084),
 (7868, 0.24317084),
 (7869, 0.24317084),
 (7870, 0.24317084),
 (7871, 0.24317084),
 (7872, 0.24317084),
 (7874, 0.24317084),
 (7875, 0.24317084),
 (7876, 0.24317084),
 (7878, 0.24317084),
 (7867, 0.24317084),
 (8527, 0.24317084),
 (7983, 0.24317084),
 (7984, 0.24317084),
 (7985, 0.24317084),
 (7986, 0.24317084),
 (7987, 0.24317084),
 (7988, 0.24317084),
 (7989, 0.24317084),
 (7990, 0.24317084),
 (16, 0.24317084),
 (8077, 0.24317084),
 (8078, 0.24317084),
 (8079, 0.24317084),
 (8080, 0.24317084),
 (8081, 0.24317084),
 (8082

In [ ]:
session.close()
engine.dispose()